# R68 MCMC

Use MCMC to estimate yield model parameters for R68 data

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#Set up notebook and load some R68 constants (V, eps, etc.)
exec(open("nb_setup.py").read())#Is there a better way to do this?
from constants import *

../mplstyles
3.1.1
/data/chocula/mast/cdms/analysis/run068/R68_paper2019/mplstyles


In [3]:
#Load the data
import R68_load as r68

meas=r68.load_measured()
g4=r68.load_G4(load_frac=0.1)
cap=r68.load_simcap(file='/data/chocula/villaa/cascadeSimData/si28_R68_400k.pkl', rcapture=0.161, load_frac=0.1)

Loading Measured Data...
(480634,)
(174510,)
Loading Geant4 Data...
(528848, 7)
(129555, 7)
Loading NRs...
1.1  min
Loading ERs...
0.3  min
Loading (n,gamma) Data...
400000


In [52]:
#Energy binning
Emax = 2000 #eVee
Ebins=np.linspace(0,Emax,201)

#Changed on 14 Feb 2020 for speed
#Emax = 1000 #eVee 
#Ebins=np.linspace(0,Emax,101)
#Ebin_ctr=(Ebins[:-1]+Ebins[1:])/2

#Measured spectra
N_meas_PuBe,_ = np.histogram(meas['PuBe']['E'],bins=Ebins)
N_meas_Bkg,_ = np.histogram(meas['Bkg']['E'],bins=Ebins)

tlive_PuBe = meas['PuBe']['tlive']
tlive_Bkg = meas['Bkg']['tlive']
#We'll scale everything to the PuBe live time and work with counts, not rate, to get the Poisson stats right

N_meas_Bkg_scaled = N_meas_Bkg * tlive_PuBe/tlive_Bkg

#Estimate of counts due to PuBe
N_meas = N_meas_PuBe - N_meas_Bkg_scaled

In [53]:
#Import yield models
import R68_yield as Yield
import R68_spec_tools as spec

In [54]:
Y=Yield.Yield('Lind',[0.15])
print(Y.models)

['Lind: Lindhard', 'Chav: Chavarria', 'Sor: Sorenson', 'Damic: Extrapolated Damic model']


In [55]:
#Define likelihood functions
from scipy.special import factorial, gamma, loggamma

#Poisson likelihood of measuring k given expected mean of lambda
def pois_likelihood(k, lamb):
    return (lamb**k)*np.exp(-lamb)/gamma(k+1.)

#Poisson log-likelihood
#k: observed counts
#lamb: expected (model) counts
def ll_pois(k, lamb):   
    if np.sum(lamb<=0):
        return -np.inf
    
    return np.sum(k*np.log(lamb) - lamb - loggamma(k+1.))

#Normal log-likelihood, limit of Poisson for large lambda
#k: observed counts
#lamb: expected (model) counts
def ll_norm(k,lamb):
    if np.sum(lamb<=0):
        return -np.inf
    
    return np.sum(-0.5*np.log(2*np.pi*lamb) - (k-lamb)**2/(2*lamb))

In [56]:
#Log of flat prior functions
#theta: array of parameter values
#bounds: array of parameter bounds. shape should be len(theta)x2
def lp_flat(theta, bounds):
    #for itheta,ibounds in zip(theta,bounds):
    #    if not (ibounds[0] < itheta < ibounds[1]):
    #        return -np.inf
        
    #return 0.0
    
    if (np.array(bounds)[:,0]<theta).all() and (theta<np.array(bounds)[:,1]).all():
        return 0.0
    return -np.inf

#Log of normal prior distribution
#theta: parameter value(s)
#mu: parameter prior distribution mean(s)
#sigma: paramter prior distribution sigma(s)
def lp_norm(theta, mu, sigma):
    return np.sum(-0.5*((theta-mu)/sigma)**2 - np.log(sigma)-0.5*np.log(2*np.pi))

In [57]:
#Calculate Log probability, log(likelihood*prior)
#
#model: pre-defined Yield model
#theta: array of fit parameters (yield_par0, yield_par1, ...,  F_NR, scale_g4, scale_ng, ...)
#theta_bounds: paramter bounds, shape should be len(theta)x2
#spec_bounds: range of bin numbers in spectrum to consider. The analysis range is [bin_low,bin_high)
#likelihood: Likelihood function, either 'Pois' or 'Norm'

def calc_log_prob(model='Lind',theta=[0.2, 1, 1, 1], theta_bounds=((0,1),(0,10),(0,10),(0,10)), spec_bounds=(5,101),
                  likelihood='Pois'):

    #Access the global data we loaded
    global N_meas, tlive_PuBe, g4, cap, Y
    
    ############
    #Set some local variables
    F_NR=None
    scale_g4=None
    scale_ng=None
    NR=None
    ER=None
    NG=None
    
    if model=='Lind':
        nYpar=1
    elif (model=='Chav' or model=='Sor'):
        nYpar=2
    elif (model=='Damic'):
        nYpar=0
    else:
        print('Error: Yield model not defined.')
        return None
    
    Y.model=model
    Y.pars=theta[:nYpar]
    F_NR=theta[nYpar]
    scale_g4=theta[nYpar+1]
    scale_ng=theta[nYpar+2]
    
    
    #Calculate the (log)prior first since we may not need to calculate the likelihood
    lp=lp_flat(theta, theta_bounds)
    if not np.isfinite(lp):
        return -np.inf
        
    
    ##########
    #Build the spectra
    #NR,ER,NG=spec.buildSimSpectra_ee(Ebins=Ebins, Evec_nr=g4['NR']['E'], Evec_er=g4['ER']['E'], Evec_ng=cap['E'], dEvec_ng=cap['dE'], 
                                         #Yield=Y, F_NR=F_NR, scale_g4=scale_g4, scale_ng=scale_ng, doDetRes=True, seed=1)

    #Avg spectra is slower more stable
    NR,ER,NG=spec.buildAvgSimSpectra_ee(Ebins=Ebins, Evec_nr=g4['NR']['E'], Evec_er=g4['ER']['E'], Evec_ng=cap['E'], dEvec_ng=cap['dE'],
                                        Yield=Y, F_NR=F_NR, scale_g4=scale_g4, scale_ng=scale_ng, doDetRes=True, fpeak=1)


    #Total counts for PuBe live time
    #Uncertainty will be sqrt(N)
    N_pred = (NR/g4['NR']['tlive'] + ER/g4['ER']['tlive'] + NG/cap['tlive'])*tlive_PuBe

    ##########
    #Calculate the log probability = log prior + log likelihood
    ll=None
    
    if likelihood=='Norm':
        ll = ll_norm(N_meas[slice(*spec_bounds)],N_pred[slice(*spec_bounds)])
    elif likelihood=='Pois':
        ll = ll_pois(N_meas[slice(*spec_bounds)],N_pred[slice(*spec_bounds)])
    else:
        print('Error: Bad likelihood')
        return None
    
    if not np.isfinite(ll):
        return -np.inf
    
    return lp + ll

# Lindhard Fit

In [ ]:
import emcee
from multiprocessing import Pool

#Lindhard fit
# theta = k, sim_scale, F_NR
labels_l = ['k', 'sim scale', 'F_{NR}']

Y=Yield.Yield('Lind',[0.2])

#Wrapper help
def LindFit_helper(theta):
    return calc_log_prob(model='Lind', theta=theta, theta_bounds=((0,1),(0,10),(0,5)),
                         spec_bounds=(5,101), likelihood='Pois')

In [ ]:
nwalkers_l = 8
ndim_l = 3
nstep_l=5000
#guesses_l = np.array([0.18, 1.6, 3.8]) + 1e-3 * np.random.randn(nwalkers_l, ndim_l)
guesses_l = np.array([0.18, 1.6, 3.8]) + np.array([1e-2, 1, 0.5]) * np.random.randn(nwalkers_l, ndim_l)

with Pool(processes=20) as pool: #Can fail depending on current memory usage of other processes...
    sampler_l = emcee.EnsembleSampler(nwalkers_l, ndim_l, LindFit_helper, pool=pool)
    sampler_l.run_mcmc(guesses_l, nstep_l, progress=True);

In [ ]:
#Save this work
import pickle as pkl
import os

saveMCMC=True

if saveMCMC:
    ifile = 0
    fname='data/mcmc_Lind_{0}walk_{1}step_pois_v{2}.pkl'.format(nwalkers_l,nstep_l,ifile+1)
    while os.path.exists(fname):
        ifile += 1
        fname='data/mcmc_Lind_{0}walk_{1}step_pois_v{2}.pkl'.format(nwalkers_l,nstep_l,ifile+1)
        
    print(fname)
    saveFile = open(fname, 'wb')
    pkl.dump(sampler_l,saveFile)
    pkl.dump(guesses_l,saveFile)
    pkl.dump(labels_l,saveFile)
    saveFile.close()

In [ ]:
#Look at the chain of parameter values
fig, axes = plt.subplots(ndim_l, figsize=(10, 7), sharex=True)
samples_l = sampler_l.get_chain()
for i in range(ndim_l):
    ax = axes[i]
    ax.plot(samples_l[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples_l))
    #ax.set_ylim(0, 5)
    ax.set_ylabel(labels_l[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
#Get the sample autocorrelation time
tau_l=sampler_l.get_autocorr_time()
print(tau_l)
avgtau_l=round(np.average(tau_l))
print(avgtau_l)

In [ ]:
#Discard a few times tau as burn-in and thin by tau/2
avgtau_l=90
flat_samples_l = sampler_l.get_chain(discard=int(2.*avgtau_l), thin=int(round(avgtau_l/2.)), flat=True)
#flat_samples_l = sampler.get_chain(discard=40, flat=True)
print(flat_samples_l.shape)

In [ ]:
import corner
fig = corner.corner(flat_samples_l, labels=labels_l, quantiles=[0.16, 0.5, 0.84], show_titles=True, title_fmt='0.3f');

In [ ]:
from IPython.display import display, Math

for i in range(ndim_l):
    mcmc = np.percentile(flat_samples_l[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{+{2:.3f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labels_l[i])
    display(Math(txt))

In [ ]:
print(flat_samples_l[0])

In [ ]:
#Plot best fit yield
fig, ax = plt.subplots(1,1)

Er_plot=np.linspace(100,3e3,100)

inds = np.random.randint(len(flat_samples_l), size=100)
for ind in inds:
    thetai = flat_samples_l[ind][0]
    Y=Yield.Yield('Lind',[thetai])
    plt.plot(Er_plot, Y.calc(Er_plot), "C3", alpha=0.1)

ax.set_xlabel('Er [eV]')
ax.set_ylabel('Y')
ax.set_ylim(0.1,0.3);

#plt.legend()

# Chavarria Fit

In [ ]:
import emcee
from multiprocessing import Pool

#Chavarria fit
# theta = k, a, sim_scale, F_NR

# k: Lindhard k factor [unitless]
# ainv: Inverse Chavaria cutoff factor [1/eV]
#labels_c = ['k', 'ainv', 'sim scale', 'F_{NR}']

Y=Yield.Yield('Chav',[0.2,10])

#Wrapper help
#def ChavFit_helper(theta):
#    return calc_log_prob(model='Chav', theta=theta, theta_bounds=((0,1),(0,100),(0,10),(0,5)),
#                         spec_bounds=(5,101), likelihood='Pois')


# theta = k, a, F_NR, scale_g4, scale_ng,
labels_c = [r'k', r'$a^{-1}$', r'$F_{NR}$', r'$scale_{G4}$', r'$scale_{ng}$']

#Test full range fit
def ChavFit_helper(theta):
    return calc_log_prob(model='Chav', theta=theta, theta_bounds=((0,1),(0,1000),(0,10),(0,10),(0,10)),
                         spec_bounds=(5,201), likelihood='Pois')

In [ ]:
#nwalkers_c = 8
#ndim_c = 4
#nstep_c = 5000

#guesses_c = np.array([0.18, 10., 1.6, 3.8]) + np.array([1e-2, 1., 1, 0.5]) * np.random.randn(nwalkers_c, ndim_c)

#with Pool(processes=20) as pool: #Can fail depending on current memory usage of other processes...
#    sampler_c = emcee.EnsembleSampler(nwalkers_c, ndim_c, ChavFit_helper, pool=pool)
#    sampler_c.run_mcmc(guesses_c, nstep_c, progress=True);
    
nwalkers_c = 16
ndim_c = 5
nstep_c = 5000

guesses_c = np.array([0.18, 10, 3.0, 1.0, 1.0]) + np.array([1e-2, 1, 1, 0.1, 0.1]) * np.random.randn(nwalkers_c, ndim_c)

with Pool(processes=20) as pool: #Can fail depending on current memory usage of other processes...
    sampler_c = emcee.EnsembleSampler(nwalkers_c, ndim_c, ChavFit_helper, pool=pool)
    sampler_c.run_mcmc(guesses_c, nstep_c, progress=True);

In [ ]:
#Save this work
import pickle as pkl
import os

saveMCMC=True

if saveMCMC:
    ifile = 0
    fname='data/mcmc_Chav_{0}walk_{1}step_pois_v{2}.pkl'.format(nwalkers_c,nstep_c,ifile+1)
    while os.path.exists(fname):
        ifile += 1
        fname='data/mcmc_Chav_{0}walk_{1}step_pois_v{2}.pkl'.format(nwalkers_c,nstep_c,ifile+1)
        
    print(fname)
    saveFile = open(fname, 'wb')
    
    results={'sampler':sampler_c, 'guesses': guesses_c, 'labels':labels_c}
    
    pkl.dump(results,saveFile)
    saveFile.close()

# Sorenson Fit

In [58]:
import emcee
from multiprocessing import Pool

#Sorenson fit
# theta = k, q, sim_scale, F_NR
# k: Lindhard k factor [unitless]
# q: Cutoff energy, in units of Lindhard epsilon (eps = 11.5*Er/1000*Z**(-7./3))
#labels_s = ['k', 'q', 'sim scale', 'F_{NR}']

Y=Yield.Yield('Sor',[0.2,2e-3])

#Wrapper help
#def SorFit_helper(theta):
#    return calc_log_prob(model='Sor', theta=theta, theta_bounds=((0,1),(0,3e-2),(0,10),(0,5)),
#                         spec_bounds=(5,101), likelihood='Pois')

# theta = k, q, F_NR, scale_g4, scale_ng,
labels_s = [r'k', r'q', r'$F_{NR}$', r'$scale_{G4}$', r'$scale_{ng}$']

#Wrapper help
#def SorFit_helper(theta):
#    return calc_log_prob(model='Sor', theta=theta, theta_bounds=((0,1),(0,3e-2),(0,10),(0,10),(0,10)),
#                         spec_bounds=(5,101), likelihood='Pois')

#Test full range fit
def SorFit_helper(theta):
    E_lim_min=50 #eVee
    E_lim_max=1.75e3 #eVee
    spec_bounds=(np.digitize(E_lim_min,Ebins)-1,np.digitize(E_lim_max,Ebins)-1)
                 
    return calc_log_prob(model='Sor', theta=theta, theta_bounds=((0,1),(0,3e-2),(0,10),(0,10),(0,10)),
                         spec_bounds=spec_bounds, likelihood='Pois')

In [59]:
#nwalkers_s = 8
#ndim_s = 4
#nstep_s = 5000

#guesses_s = np.array([0.18, 2e-3, 1.6, 3.8]) + np.array([1e-2, 1e-4, 1, 0.5]) * np.random.randn(nwalkers_s, ndim_s)

nwalkers_s = 16
ndim_s = 5
nstep_s = 5000

#guesses_s = np.array([0.18, 2e-3, 3.0, 1.0, 1.0]) + np.array([1e-2, 1e-4, 1, 0.1, 0.1]) * np.random.randn(nwalkers_s, ndim_s)

#Sample priors uniformly
tbs=np.array(((0,1),(0,3e-2),(0,10),(0,10),(0,10)))
guesses_s=(np.array(tbs)[:,1]-np.array(tbs)[:,0])*np.random.random_sample((nwalkers_s, ndim_s))+np.array(tbs)[:,0]

with Pool(processes=20) as pool: #Can fail depending on current memory usage of other processes...
    sampler_s = emcee.EnsembleSampler(nwalkers_s, ndim_s, SorFit_helper, pool=pool)
    sampler_s.run_mcmc(guesses_s, nstep_s, progress=True);

100%|██████████| 5000/5000 [5:39:18<00:00,  4.07s/it]  


In [60]:
#Save this work
import pickle as pkl
import os

saveMCMC=True

if saveMCMC:
    ifile = 0
    fname='data/mcmc_Sor_{0}walk_{1}step_pois_v{2}.pkl'.format(nwalkers_s,nstep_s,ifile+1)
    while os.path.exists(fname):
        ifile += 1
        fname='data/mcmc_Sor_{0}walk_{1}step_pois_v{2}.pkl'.format(nwalkers_s,nstep_s,ifile+1)
        
    print(fname)
    saveFile = open(fname, 'wb')
    
    results={'sampler':sampler_s, 'guesses': guesses_s, 'labels':labels_s}
    
    pkl.dump(results,saveFile)
    saveFile.close()

data/mcmc_Sor_16walk_5000step_pois_v9.pkl


In [ ]:
#Look at the chain of parameter values
fig, axes = plt.subplots(ndim_s, figsize=(10, 7), sharex=True)
samples_s = sampler_s.get_chain()
for i in range(ndim_s):
    ax = axes[i]
    ax.plot(samples_s[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples_s))
    #ax.set_ylim(0, 5)
    ax.set_ylabel(labels_s[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
#Get the sample autocorrelation time
tau_s=sampler_s.get_autocorr_time()
print(tau_s)
avgtau_s=round(np.average(tau_s))
print(avgtau_s)

In [ ]:
#Discard a few times tau as burn-in and thin by tau/2
avgtau_s=200
flat_samples_s = sampler_s.get_chain(discard=int(2.*avgtau_s), thin=int(round(avgtau_s/2.)), flat=True)
print(flat_samples_s.shape)

In [ ]:
import corner
fig = corner.corner(flat_samples_s, labels=labels_s, quantiles=[0.16, 0.5, 0.84], show_titles=True, title_fmt='0.3f');

## Lind Fit Including Systematics
Let's try to include nuisance factors for things like measured energy calibration and cut efficiencies

In [ ]:
#Calculate Log probability, log(likelihood*prior)
#
#model: pre-defined Yield model
#theta: array of fit parameters: here its (yield pars, sim scale, F_NR, calibration scale)
#theta_bounds: paramter bounds, shape should be len(theta)x2
#spec_bounds: range of bin numbers in spectrum to consider. The analysis range is [bin_low,bin_high)
#likelihood: Likelihood function, either 'Pois' or 'Norm'

def calc_log_prob_wnuis(model='Lind',theta=[0.2, 1.5, 2.4, 1.0], theta_bounds=((0,1),(0,10),(0,5),(0.5,1.5)), 
                        spec_bounds=(5,101), likelihood='Pois'):

    #Access the global data we loaded
    global meas, tlive_PuBe, g4, cap, Y
    
    ############
    #Set some local variables
    F_NR=None
    scale_g4=None
    scale_ng=None
    NR=None
    ER=None
    NG=None
    
    if model=='Lind':
        nYpar=1
    elif (model=='Chav' or model=='Sor'):
        nYpar=2
    elif (model=='Damic'):
        nYpar=0
    else:
        print('Error: Yield model not defined.')
        return None
    
    Y.model=model
    Y.pars=theta[:nYpar]
    F_NR=theta[nYpar+1]
    scale_g4=theta[nYpar]
    scale_ng=scale_g4
    cal_scale=theta[nYpar+2]
    
    ##########
    #Build the measured spectra    
    N_meas_PuBe,_ = np.histogram(cal_scale*meas['PuBe']['E'],bins=Ebins)
    N_meas_Bkg,_ = np.histogram(cal_scale*meas['Bkg']['E'],bins=Ebins)

    tlive_PuBe = meas['PuBe']['tlive']
    tlive_Bkg = meas['Bkg']['tlive']
    
    #We'll scale everything to the PuBe live time and work with counts, not rate, to get the Poisson stats right
    N_meas_Bkg_scaled = N_meas_Bkg * tlive_PuBe/tlive_Bkg

    #Estimate of counts due to PuBe
    N_meas = N_meas_PuBe - N_meas_Bkg_scaled
    
    
    #Calculate the (log)prior first since we may not need to calculate the likelihood
    lp=lp_flat(theta[:nYpar+2], theta_bounds[:nYpar+2])
    lp+=lp_norm(theta[nYpar+2], 1, 0.1)#normal distribution for cal_scale
    if not np.isfinite(lp):
        return -np.inf
        
    
    ##########
    #Build the simulated spectra
    #Avg spectra is more stable
    NR,ER,NG=spec.buildAvgSimSpectra_ee(Ebins=Ebins, Evec_nr=g4['NR']['E'], Evec_er=g4['ER']['E'], Evec_ng=cap['E'], dEvec_ng=cap['dE'],
                                        Yield=Y, F_NR=F_NR, scale_g4=scale_g4, scale_ng=scale_ng, doDetRes=True)


    #Total counts for PuBe live time
    #Uncertainty will be sqrt(N)
    N_pred = (NR/g4['NR']['tlive'] + ER/g4['ER']['tlive'] + NG/cap['tlive'])*tlive_PuBe

    ##########
    #Calculate the log probability = log prior + log likelihood
    ll=None
    
    if likelihood=='Norm':
        ll = ll_norm(N_meas[slice(*spec_bounds)],N_pred[slice(*spec_bounds)])
    elif likelihood=='Pois':
        ll = ll_pois(N_meas[slice(*spec_bounds)],N_pred[slice(*spec_bounds)])
    else:
        print('Error: Bad likelihood')
        return None
    
    if not np.isfinite(ll):
        return -np.inf
    
    return lp + ll

In [ ]:
import emcee
from multiprocessing import Pool

#Lindhard fit
# theta = k, sim_scale, F_NR, cal_scale
labels_l = ['k', 'sim scale', 'F_{NR}', 'cal scale']

Y=Yield.Yield('Lind',[0.2])

#Wrapper help
def LindFit_wnuis_helper(theta):
    return calc_log_prob_wnuis(model='Lind', theta=theta, theta_bounds=((0,1),(0,10),(0,5),(0,10)),
                         spec_bounds=(5,101), likelihood='Pois')

In [ ]:
nwalkers_l = 8
ndim_l = 4
nstep_l=5000
#guesses_l = np.array([0.18, 1.6, 3.8, 1.0]) + 1e-3 * np.random.randn(nwalkers_l, ndim_l)
guesses_l = np.array([0.18, 1.6, 3.8, 1.0]) + np.array([1e-2, 1, 0.5, 1e-1]) * np.random.randn(nwalkers_l, ndim_l) #Set the randoms scale better

with Pool(processes=20) as pool: #Can fail depending on current memory usage of other processes...
    sampler_l = emcee.EnsembleSampler(nwalkers_l, ndim_l, LindFit_wnuis_helper, pool=pool)
    sampler_l.run_mcmc(guesses_l, nstep_l, progress=True);

In [ ]:
#Save this work
import pickle as pkl
import os

saveMCMC=True

if saveMCMC:
    ifile = 0
    fname='data/mcmc_Lind_wcalscale_{0}walk_{1}step_pois_v{2}.pkl'.format(nwalkers_l,nstep_l,ifile+1)
    while os.path.exists(fname):
        ifile += 1
        fname='data/mcmc_Lind_wcalscale_{0}walk_{1}step_pois_v{2}.pkl'.format(nwalkers_l,nstep_l,ifile+1)
        
    print(fname)
    saveFile = open(fname, 'wb')
    pkl.dump(sampler_l,saveFile)
    pkl.dump(guesses_l,saveFile)
    saveFile.close()

In [ ]:
#Look at the chain of parameter values
fig, axes = plt.subplots(ndim_l, figsize=(10, 7), sharex=True)
samples_l = sampler_l.get_chain()
for i in range(ndim_l):
    ax = axes[i]
    ax.plot(samples_l[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples_l))
    #ax.set_ylim(0, 5)
    ax.set_ylabel(labels_l[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
#Get the sample autocorrelation time
tau_l=sampler_l.get_autocorr_time()
print(tau_l)
avgtau_l=round(np.average(tau_l))
print(avgtau_l)

In [ ]:
#Discard a few times tau as burn-in and thin by tau/2
avgtau_l=150
flat_samples_l = sampler_l.get_chain(discard=int(2.*avgtau_l), thin=int(round(avgtau_l/2.)), flat=True)
#flat_samples_l = sampler.get_chain(discard=40, flat=True)
print(flat_samples_l.shape)

In [ ]:
import corner

fig = corner.corner(
    flat_samples_l, labels=labels_l
);

In [ ]:
plt.plot(samples_l[:, :, 0].flatten(),samples_l[:, :, 3].flatten(),'.',alpha=0.01)
plt.gca().set_ylim(0.9990,0.9995)